## Section 1: Import Libraries and load cifar10 dataset.

In [10]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from torch.utils.data import SubsetRandomSampler
#import additional libraries if needed

In [11]:
def load_cifar10_data(batch_size):
# load both training and test datasets, and transform them to tensors.
  transform = transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform)
  trainset.data = trainset.data[:30000]
  trainset.targets = trainset.targets[:30000]
  print("train",len(trainset))
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
  #testset.data = testset.data[:2000]
  #testset.targets = testset.targets[:2000]
  print("test",len(testset))
  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)
  
  return trainset, testset, trainloader, testloader

In [12]:
batch_size = 4
trainset, testset, trainloader, testloader = load_cifar10_data(batch_size)
classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Check the dimensions of a batch:
for images, labels in trainloader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Files already downloaded and verified
train 30000
Files already downloaded and verified
test 10000
Image batch dimensions: torch.Size([4, 3, 32, 32])
Image label dimensions: torch.Size([4])


## Section 2: Load Pre-trained Model 

## Section 2.1: ALEXNET

In [13]:
from torchvision import models

# Load the pre-trained VGG16 model
model = torchvision.models.vgg16(pretrained=True)



# Print modified VGG model

print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
import torch.nn as nn

for param in model.parameters():
    param.requires_grad = False


# Modify first max pooling layer
model.avgpool = nn.AdaptiveAvgPool2d((6, 6))
model.features = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(64, 192, kernel_size=5, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(192, 384, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
)

# Replace the output layer with your own output layer
output_layer_index = -1
model.classifier = nn.Sequential(
    nn.Dropout(),
    nn.Linear(256 * 6 * 6, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 10),
)







## Section 3: Define optimizer and loss criterion

In [15]:
# Hyperparameters
random_seed = 42
learning_rate = 0.001
num_epochs = 10

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# loss
loss = nn.CrossEntropyLoss()

In [16]:
"""DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', DEVICE)
# Set my pretrained model to be run on GPU
model = model.to(DEVICE)"""

'DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\nprint(\'Device:\', DEVICE)\n# Set my pretrained model to be run on GPU\nmodel = model.to(DEVICE)'

## Section 4: Train the model

In [17]:
def train_model(trainset, num_epochs, model, optimizer, loss):
    # split data into 10 folds
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    fold_losses = []
    fold_val_accs = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(trainset)):
        print(f"Fold {fold + 1}")

        # get fold data
        train_fold = torch.utils.data.Subset(trainset, train_idx)
        val_fold = torch.utils.data.Subset(trainset, val_idx)

        # create data loaders
        batch_size = 32
        trainloader = torch.utils.data.DataLoader(train_fold, batch_size=batch_size,
                                                  shuffle=True, num_workers=2)
        valloader = torch.utils.data.DataLoader(val_fold, batch_size=batch_size,
                                                shuffle=False, num_workers=2)

        # train for num_epochs epochs
        for epoch in range(num_epochs):
            running_loss = 0.0

            for i, (inputs, labels) in enumerate(trainloader):
                optimizer.zero_grad()

                outputs = model(inputs)
                loss_value = loss(outputs, labels)
                loss_value.backward()
                optimizer.step()

                running_loss += loss_value.item()

            # calculate validation accuracy
            correct = 0
            total = 0
            with torch.no_grad():
                for val_inputs, val_labels in valloader:
                    val_outputs = model(val_inputs)
                    _, predicted = torch.max(val_outputs.data, 1)
                    total += val_labels.size(0)
                    correct += (predicted == val_labels).sum().item()

            val_acc = correct / total
            fold_val_accs.append(val_acc)

            print(f"Epoch {epoch + 1}: Loss={running_loss:.4f}, Validation Accuracy={val_acc:.4f}")

        fold_losses.append(running_loss)

    # calculate final validation accuracy
    mean_val_acc = sum(fold_val_accs) / len(fold_val_accs)
    print(f"Final Validation Accuracy: {mean_val_acc:.4f}")

In [18]:
train_model(trainset, num_epochs, model, optimizer, loss)

Fold 1
Epoch 1: Loss=1942.8889, Validation Accuracy=0.0983
Epoch 2: Loss=1901.2516, Validation Accuracy=0.1710
Epoch 3: Loss=1674.9257, Validation Accuracy=0.2807
Epoch 4: Loss=1497.7879, Validation Accuracy=0.3430
Epoch 5: Loss=1377.7948, Validation Accuracy=0.3977
Epoch 6: Loss=1253.6155, Validation Accuracy=0.4473
Epoch 7: Loss=1158.1295, Validation Accuracy=0.5060
Epoch 8: Loss=1056.5567, Validation Accuracy=0.5520
Epoch 9: Loss=973.1740, Validation Accuracy=0.5950
Epoch 10: Loss=902.1499, Validation Accuracy=0.5750
Fold 2
Epoch 1: Loss=854.4022, Validation Accuracy=0.6507
Epoch 2: Loss=792.8646, Validation Accuracy=0.6597
Epoch 3: Loss=739.7081, Validation Accuracy=0.6787
Epoch 4: Loss=688.1162, Validation Accuracy=0.6837
Epoch 5: Loss=628.9882, Validation Accuracy=0.7147
Epoch 6: Loss=587.7777, Validation Accuracy=0.7017
Epoch 7: Loss=543.2850, Validation Accuracy=0.7333
Epoch 8: Loss=500.2359, Validation Accuracy=0.7067
Epoch 9: Loss=467.9998, Validation Accuracy=0.7340
Epoch 10

## Section 5: Test the model

Write a function to test the model using testloader.

In [19]:
dataiter = iter(testloader)
images, labels = next(dataiter)

In [20]:
#Test for the entire dataset

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 76 %
